<a href="https://colab.research.google.com/github/artoowang/ray-tracer-colab/blob/main/Cython%2BC_Workflow_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cython+C Workflow Utilities

In [37]:
import os
import re
import importlib.util
from typing import Tuple

def _extract_reload_number(ext_path: str) -> int:
  """Returns the reload number of the given extension path.

  E.g.,
  _extract_reload_number('/usr/local/lib/python3.7/dist-packages/helloworld.reload6.so')
  returns 6.
  """
  result = re.search(r'\.reload([0-9]*)\.so', ext_path)
  if not result:
    raise ValueError(
        f'Failed to extract reload number from module path: {ext_path}')
  return int(result[1])


def _find_latest_reload_extension(ext_name: str) -> Tuple[int, str]:
  """Returns the reload number and path of the latest extension |ext_name|.

  Returns (reload number, extension path) tuple.
  """
  reload_module_pattern = f'{ext_name}.reload*.so'
  reload_module_paths = !ls -1 {reload_module_pattern} 2> /dev/null
  if len(reload_module_paths) == 0:
    return (0, None)
  reload_number_modules = [(_extract_reload_number(module_path), module_path)
                           for module_path in reload_module_paths]
  return max(reload_number_modules)


def _build(setup_path, ext_name=None):
  """Builds the extension with the given setup file path and extension name.

  Args:
    setup_path: The path to a Python file for Python module setup.
    ext_name: Optional. When given, it is used as the Python module name.
      Otherwise, the module name is derived from the setup Python file path with
      the pattern {module_name}_setup.py. It is assumed that the setup file is
      configured to create a module with this name.

  Returns:
    The path to the new, renamed module with reload<N>, where N is a new
    reload number above all the existing reload modules.
    E.g., if we already have helloworld.reload6.so and helloworld.reload7.so,
    then this will build helloworld.reload8.so and returns the path to it.
  """
  # Extract extension name from |setup_path|.
  result = re.match(r'(.*/)?([^/]+)_setup.py', setup_path)
  if result:
    ext_name = result[2]
  print(f'Build module {ext_name} ...')

  next_reload_number = _find_latest_reload_extension(ext_name)[0] + 1
  print(f'Next reload number: {next_reload_number}')
  # Use build_ext --inplace to build the module in the working directory.
  !python3 {setup_path} build_ext --inplace
  new_module_pattern = f'{ext_name}.cpython-*.so'
  new_reload_module_path = f'{ext_name}.reload{next_reload_number}.so'
  !mv {new_module_pattern} {new_reload_module_path}
  if not os.path.exists(new_reload_module_path):
    raise ValueError(f'Failed to create module at {new_reload_module_path}')
  return new_reload_module_path


def _reload_extension(ext_name: str):
  """Reloads the extension |ext_name|.

  Returns the reloaded extension module.
  """
  _, ext_path = _find_latest_reload_extension(ext_name)
  if not ext_path:
    raise ValueError(f'Cannot find extension {ext_name}.') 
  print(f'Loading extension {ext_path} ...')
  spec = importlib.util.spec_from_file_location(ext_name, ext_path)
  module = importlib.util.module_from_spec(spec)
  spec.loader.exec_module(module)
  return module

def build_and_reload_extension(ext_name: str):
  """Builds and reloads the extension |ext_name|.

  Returns the reloaded extension module.
  """
  _build(f'{ext_name}_setup.py')
  return _reload_extension(ext_name)

# Example

### Cython setup

In [2]:
%%file example_mod_setup.py

from distutils.core import setup  
from distutils.extension import Extension  
from Cython.Build import cythonize
import sys

# It is important to make sure the extension name, .pyx file, and this setup
# file names are all consistent with each other. E.g., to build a module called
# 'example_mod', the extension name should be 'example_mod', .pyx file should be
# example_mod.pyx, and the setup file should be example_mod_setup.py. These are
# used in various places in the module reload utility methods.
module_name = 'example_mod'

# OpenMP setup is platform dependent.
if sys.platform == 'darwin':
  sys_dep_compile_args = [
      '-Wno-unreachable-code',
      '-Wp,-fopenmp',
  ]
  sys_dep_link_args = [
      '-Wp,-fopenmp',
      '-lomp',
  ]
else:
  sys_dep_compile_args = ['-fopenmp']
  sys_dep_link_args = ['-fopenmp']

extensions = [
  Extension(module_name, [
                f'{module_name}.pyx',
                'example.cc',
            ],
            language='c++',
            extra_compile_args=[
                '-std=c++17',
            ] + sys_dep_compile_args,
            extra_link_args=sys_dep_link_args,
           ),
]

setup(ext_modules = cythonize(extensions, language_level='3'))

Overwriting example_mod_setup.py


### C++ sources

In [38]:
%%file example.h

#ifndef EXAMPLE_H
#define EXAMPLE_H

int AddOne(int val);

#endif  // EXAMPLE_H

Overwriting example.h


In [39]:
%%file example.cc

#include "example.h"

int AddOne(int val) {
  return val + 1;
}

Overwriting example.cc


### Cython sources

In [40]:
%%file example_mod.pyx

cimport cython
from cpython cimport array

from cython.parallel import prange


cdef extern from 'example.h':
  int AddOne(int val) nogil


@cython.boundscheck(False)
def run(Py_ssize_t length):
  cdef array.array buffer = array.array('i', [i for i in range(length)])
  cdef int[:] buffer_view = buffer
  cdef Py_ssize_t i
  for i in prange(length, nogil=True):
    buffer_view[i] = AddOne(buffer_view[i])
  return list(buffer)

Overwriting example_mod.pyx


### Build and test

In [41]:
example_mod = build_and_reload('example_mod')
example_mod.run(10)

Build module example_mod ...
Next reload number: 9
Compiling example_mod.pyx because it changed.
[1/1] Cythonizing example_mod.pyx
running build_ext
building 'example_mod' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.7m -c example_mod.cpp -o build/temp.linux-x86_64-3.7/example_mod.o -std=c++17 -fopenmp
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]